In [1]:
import pandas as pd
import copy
from janome.tokenizer import Tokenizer
from gensim.models import word2vec
from sklearn.linear_model import LogisticRegression
from sklearn import linear_model
from sklearn.metrics import (accuracy_score, confusion_matrix, f1_score,
                             precision_score, recall_score, roc_auc_score)
from sklearn.model_selection import train_test_split
import requests
import numpy as np
from bs4 import BeautifulSoup
from requests.exceptions import Timeout
import time
import re
import urllib.request
from lxml import html
import openpyxl as op
import math

記事特徴を作成

In [12]:
file = 'data/news/〇〇.xlsx' #アクセスデータ

recommend = pd.read_excel(file, sheet_name=0)
access = pd.read_excel(file, sheet_name=1)
user = pd.read_excel(file, sheet_name=2)
user = user.rename(columns={'ID（ハッシュ値）': 'user'})

In [13]:
ar_list = [] #記事の集合

#記事集合を作る
for i in range(len(recommend)):
    ar = recommend['関連記事一覧'][i].split(',')
    for j in range(len(ar)):
        ar_list.append(ar[j])

ar_list = list(set(ar_list)) #重複削除
del ar_list[0]

In [14]:
#Dataframeを作成

article = pd.DataFrame(ar_list)
article = article.rename(columns={0: 'url'})
article = article.assign(title = 'xxx', key = 'xxx', genre = 'xxx')

In [17]:
#テスト

# 1.ログインページにアクセスする
url_login = "https://kumanichi.com/user/login"
session = requests.session()
print('login1')
# ログインページへのアクセス完了
req_before_login = session.get(url_login)
print('login2')
# ログインするための情報を準備する
login_data = {
   'mail': 'kumadai.kumanichi@gmail.com',
   'pass': 'kumadai3442111'
}

# ログインするためにcsrfトークンが必要となるため情報を取得
bs = BeautifulSoup(req_before_login.text, 'html.parser')
csrf_token = bs.find(
   attrs={'name':'form_build_id'}).get('value')
login_data['form_build_id'] = csrf_token
token2 = bs.find(attrs={'name':'form_id'}).get('value')
token3 = bs.find(attrs={'name':'op'}).get('value')
login_data['form_id'] = token2
login_data['op'] = token3
print('login3')

# 2. ログインページで認証を行い、管理者ページへ遷移する
req_after_login = session.post(url_login, data=login_data)
print('login4')

for url in range(len(article)):
# for url in range(15):
   print('----------------------------')
   print('{0}/{1}'.format(url, len(article)))
   time.sleep(2.0)
   j = article.iat[url, 0]
   print(j)
   html = session.get("https://kumanichi.com" + str(j)).text
   soup = BeautifulSoup(html, "html.parser")

   #共同通信か判定
   ex = soup.find('p', class_="text-week")
   print('1')
   if(str(ex) == "None"):
      print('2')
      print("記事なし")
      continue
   else:
      print('3')
      article = soup.find('p',class_="text-week").get_text

   if(str("共同通信" in str(article)) == "True"):
      print('4')
      print("共同通信なのでスキップ")
      continue

   try:
      print('5')
      try:  
         print('6')
         #ニュースタイトル取得
         data = soup.find('h1', class_="heading1").get_text()
         print(data)
         # data = data.str.strip()
         print(data)
         areas = data.split('。')
         print("取得成功", j)
         print(areas)
         article['title'][url] = areas
         print('6.1')
         #ニュースジャンル取得
         data = soup.find(language="JavaScript")
         str_data = str(data)
         x=str_data.find('var ctp1')
         y=str_data.find('var ctp2')
         print(str_data[x:y-3])
         article['genre'][url] = str_data[x:y-3]
         print('6.2')
         #カギ記事の情報を取得
         data = soup.find('h1')
         strdata = str(data)
         if 'gold' in strdata:
            print('gold')
            key = 'gold'
         elif 'silver' in strdata:
            print('silver')
            key = 'silver'
         else:
            print('free')
            key = 'free'
         article['key'][url] = key

      except AttributeError:
         print('7')
         #ニュースタイトル取得
         data = soup.find('h1', class_="heading1").get_text()
         print(data)
         # data = data.str.strip()
         print("取得成功", j)
         print(data)
         article['title'][url] = data

         #ニュースジャンル取得
         data = soup.find(language="JavaScript")
         str_data = str(data)
         x=str_data.find('var ctp1')
         y=str_data.find('var ctp2')
         print(str_data[x:y-3])
         article['genre'][url] = str_data[x:y-3]

         #カギ記事の情報を取得
         data = soup.find('h1')
         strdata = str(data)
         if 'gold' in strdata:
            print('gold')
            key = 'gold'
         elif 'silver' in strdata:
            print('silver')
            key = 'silver'
         else:
            print('free')
            key = 'free'
         article['key'][url] = key

      except Exception as e:
         print('8')
         print("記事が存在しない")
         pass

   except:
      print('9')
      print("記事が存在しない")
      pass


login1
login2
login3
login4
----------------------------
0/560
/articles/789578
1
3
5
6

アミュプラザくまもと、プレミアム商品券を販売　1万2千円分を1万円で　16、17日に計5千セット
  

アミュプラザくまもと、プレミアム商品券を販売　1万2千円分を1万円で　16、17日に計5千セット
  
取得成功 /articles/789578
['\nアミュプラザくまもと、プレミアム商品券を販売\u30001万2千円分を1万円で\u300016、17日に計5千セット\n  ']
6.1
var ctp1 =  " 熊本のニュース,熊本の経済";
6.2
gold
----------------------------
1/560
/articles/651344
1
2
記事なし
----------------------------
2/560
/articles/795080
1
3
5
6

市房ダム水位上昇、下流域住民に早めの避難呼びかけ　熊本県発表　上流域で大雨、貯水容量の半分に
  

市房ダム水位上昇、下流域住民に早めの避難呼びかけ　熊本県発表　上流域で大雨、貯水容量の半分に
  
取得成功 /articles/795080
['\n市房ダム水位上昇、下流域住民に早めの避難呼びかけ\u3000熊本県発表\u3000上流域で大雨、貯水容量の半分に\n  ']
6.1
var ctp1 =  " 熊本のニュース,防災くまもと";
6.2
gold
----------------------------
3/560
/articles/802235
1
3
5
6

「雅　細川家の歴史と美」展が来場者1万人突破　熊本県立美術館
  

「雅　細川家の歴史と美」展が来場者1万人突破　熊本県立美術館
  
取得成功 /articles/802235
['\n「雅\u3000細川家の歴史と美」展が来場者1万人突破\u3000熊本県立美術館\n  ']
6.1
var ctp1 =  " 熊本のニュース";
6.2
gold
----------------------------
4/560
/articles/785556
1
3
5
6

「強くするのに体罰はいらない」

In [18]:
# CSV ファイル (employee.csv) として出力
article.to_csv("article_202211.csv")

記事タイトルのベクトル化

In [20]:
#タイトルの謎のリスト化をstrに戻す

ar = copy.deepcopy(article)  # deep copy

for i in range(len(ar)):
    ar['title'][i] = ar['title'][i][0].strip()

In [21]:
#形態素解析を行う
jth_list = []
jth_words = []

for i in range(len(ar)):   
    t = ar['title'][i]
    jth_list.append(t)

tokenizer = Tokenizer()
for s in jth_list:
    jth_words.append(list(tokenizer.tokenize(s, wakati=True)))

In [22]:
#word2vecを施す
model = word2vec.Word2Vec(
    jth_words,
    vector_size = 5,
    min_count = 1,
    window = 5,
    epochs = 100,
    sg = 0
)

#最も単語数の多い記事を探す
len_list = []
for i in range(len(jth_words)):
    len_list.append(len(jth_words[i]))

#Word2vecのベクトル値をdataframeに格納
word = max(len_list)
size_1 = 5

for i in range(word):
    for j in range(size_1):
        ar[f'word_{i}_{j}'] = 0.0

for h in range(len(jth_words)):
    for i in range(word):
        if i >= len(jth_words[h]):
            for j in range(size_1):
                ar[f'word_{i}_{j}'][h] = 0
        else:
            wv = model.wv[jth_words[h][i]].tolist()
            for j in range(size_1):
                ar[f'word_{i}_{j}'][h] = wv[j] 



/var/folders/fz/x8gfkjkj2y97_snsf3gnnrlr0000gn/T/ipykernel_92182/1713048254.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  ar[f'word_{i}_{j}'] = 0.0
/var/folders/fz/x8gfkjkj2y97_snsf3gnnrlr0000gn/T/ipykernel_92182/1713048254.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  ar[f'word_{i}_{j}'] = 0.0
/var/folders/fz/x8gfkjkj2y97_snsf3gnnrlr0000gn/T/ipykernel_92182/1713048254.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performa

In [100]:
#平均をとる
for i in range(size_1):
    a = ar.filter(regex=f'{i}$', axis=1)
    ar[f'average_{i}'] = a.mean(axis=1)

ar = ar[['url', 'title', 'key', 'genre', 'average_0', 'average_1', 'average_2', 'average_3', 'average_4']] #カラムを減らす

/var/folders/fz/x8gfkjkj2y97_snsf3gnnrlr0000gn/T/ipykernel_42201/3785540534.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ar[f'average_{i}'] = a.mean(axis=1)
/var/folders/fz/x8gfkjkj2y97_snsf3gnnrlr0000gn/T/ipykernel_42201/3785540534.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ar[f'average_{i}'] = a.mean(axis=1)
/var/folders/fz/x8gfkjkj2y97_snsf3gnnrlr0000gn/T/ipykernel_42201/3785540534.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using 

In [101]:
ar

,url,title,key,genre,average_0,average_1,average_2,average_3,average_4
0,/articles/855554,堀江 隆臣氏（55） 地域経済再生へ、産業を支援【上天草市長選・私の公約】,gold,"var ctp1 = "" 熊本のニュース,選挙熊本"";",0.884714,0.979944,0.500595,-1.291782,0.421474
1,/articles/805313,事業決定から14年、道路拡幅いつ終わる？ 熊本市の子飼～浄行寺交差点 用地交渉に課題、早くて...,free,"var ctp1 = "" 熊本のニュース,SNSこちら編集局"";",-0.628189,0.391149,1.315344,-0.431510,0.398157
2,/articles/796893,山林が7割の美里町 「自伐型林業」に熱視線 小規模、経済と環境保全両立,gold,"var ctp1 = "" 熊本のニュース"";",-0.036666,0.762990,0.616665,-0.146576,-0.194937
3,/articles/772704,国内初の「ジャベリン」実弾射撃 大矢野原演習場で日米共同訓練 熊本県山都町,gold,"var ctp1 = "" 熊本のニュース"";",-0.406085,0.537081,0.583475,-0.567297,-0.394258
4,/articles/854331,＜とぴっく・山都町＞学校給食用に無農薬米贈呈,gold,"var ctp1 = "" 熊本のニュース"";",0.153138,1.279791,0.038954,-0.485441,0.160200
...,...,...,...,...,...,...,...,...,...
1203,/articles/774458,半導体用ガス 製造設備増強 日本エア・リキード、合志市で,gold,"var ctp1 = "" 熊本のニュース,熊本の経済"";",-0.266531,0.561183,0.092898,-0.591652,-0.247866
1204,/articles/573892,ハマグリ漁師「やっと仕事」 １カ月ぶり漁再開 熊本市沖,gold,"var ctp1 = "" 熊本のニュース"";",-0.242578,-0.028681,0.536164,-0.644769,0.096344
1205,/articles/734707,【動物のいる風景】行き場失った犬や猫に家族 ＜愛護団体「チームにゃわん」 梅崎恵美子代表 ㊤＞,gold,"var ctp1 = "" 熊本のニュース"";",0.872809,0.945743,0.941882,0.146670,0.238188
1206,/articles/835296,「墓石は減価償却の対象」 熊本市が裁判で主張 倒木で損壊の賠償めぐり,gold,"var ctp1 = "" 熊本のニュース"";",-0.135449,0.312942,0.569425,-0.288438,-0.143248


ジャンルを数値化

In [102]:
#テスト
ar['g'] = ar['genre']
ar.loc[:,'g'] = ar['g'].str.strip('var ctp1 = ') #あとで'g'は'genre'に置き換える
ar.loc[:,'g'] = ar['g'].str.strip('"')
ar.loc[:,'g'] = ar['g'].str.strip('";')
ar.loc[:,'g'] = ar['g'].str.split(',')

#最もジャンル数の多い記事を探す
g_list = []
for i in range(len(ar)):
    g_list.append(len(ar['g'][i]))

#各ジャンル名ををdataframeに格納
g_num = max(g_list)

for i in range(g_num):
    ar[f'g_{i}'] = 'xxx'

for i in range(len(ar)):
    for j in range(len(ar['g'][i])):
        ar[f'g_{j}'][i] = ar['g'][i][j]

/var/folders/fz/x8gfkjkj2y97_snsf3gnnrlr0000gn/T/ipykernel_42201/193994487.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ar[f'g_{j}'][i] = ar['g'][i][j]


In [103]:
#形態素解析を行う
jth_list = []
jth_words = []

for i in range(len(ar)):   
    t = ar['g_0'][i]
    jth_list.append(t)

tokenizer = Tokenizer()
for s in jth_list:
  jth_words.append(list(tokenizer.tokenize(s, wakati=True)))

#word2vecを施す
model = word2vec.Word2Vec(
    jth_words,
    vector_size = 5,
    min_count = 1,
    window = 5,
    epochs = 100,
    sg = 0
)

#最も単語数の多いジャンル名を探す
glen_list = []
for i in range(len(jth_words)):
    glen_list.append(len(jth_words[i]))

#Word2vecのベクトル値をdataframeに格納
word = max(glen_list)
size_1 = 5

for i in range(word):
    for j in range(size_1):
        ar[f'gword_1_{i}_{j}'] = 0.0

for h in range(len(jth_words)):
    for i in range(word):
        if i >= len(jth_words[h]):
            for j in range(size_1):
                ar[f'gword_1_{i}_{j}'][h] = 0
        else:
            wv = model.wv[jth_words[h][i]].tolist()
            for j in range(size_1):
                ar[f'gword_1_{i}_{j}'][h] = wv[j] 

for i in range(size_1):
    a = (ar.filter(regex='^gwor')).filter(regex=f'_{i}$') #始まりが「gwor」、終わりが「_0」の文字列がマッチ
    ar[f'g_ave_{i}'] = a.mean(axis=1)

/var/folders/fz/x8gfkjkj2y97_snsf3gnnrlr0000gn/T/ipykernel_42201/2282784228.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ar[f'gword_1_{i}_{j}'][h] = wv[j]
/var/folders/fz/x8gfkjkj2y97_snsf3gnnrlr0000gn/T/ipykernel_42201/2282784228.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ar[f'gword_1_{i}_{j}'][h] = 0


In [104]:
ar = ar[['url', 'key', 'genre', 'average_0', 'average_1', 'average_2','average_3', 'average_4', 'g_ave_0', 'g_ave_1', 'g_ave_2', 'g_ave_3', 'g_ave_4']]

In [105]:
ar

,url,key,genre,average_0,average_1,average_2,average_3,average_4,g_ave_0,g_ave_1,g_ave_2,g_ave_3,g_ave_4
0,/articles/855554,gold,"var ctp1 = "" 熊本のニュース,選挙熊本"";",0.884714,0.979944,0.500595,-1.291782,0.421474,-0.1108,0.121332,0.009689,-0.009051,0.037003
1,/articles/805313,free,"var ctp1 = "" 熊本のニュース,SNSこちら編集局"";",-0.628189,0.391149,1.315344,-0.431510,0.398157,-0.1108,0.121332,0.009689,-0.009051,0.037003
2,/articles/796893,gold,"var ctp1 = "" 熊本のニュース"";",-0.036666,0.762990,0.616665,-0.146576,-0.194937,-0.1108,0.121332,0.009689,-0.009051,0.037003
3,/articles/772704,gold,"var ctp1 = "" 熊本のニュース"";",-0.406085,0.537081,0.583475,-0.567297,-0.394258,-0.1108,0.121332,0.009689,-0.009051,0.037003
4,/articles/854331,gold,"var ctp1 = "" 熊本のニュース"";",0.153138,1.279791,0.038954,-0.485441,0.160200,-0.1108,0.121332,0.009689,-0.009051,0.037003
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1203,/articles/774458,gold,"var ctp1 = "" 熊本のニュース,熊本の経済"";",-0.266531,0.561183,0.092898,-0.591652,-0.247866,-0.1108,0.121332,0.009689,-0.009051,0.037003
1204,/articles/573892,gold,"var ctp1 = "" 熊本のニュース"";",-0.242578,-0.028681,0.536164,-0.644769,0.096344,-0.1108,0.121332,0.009689,-0.009051,0.037003
1205,/articles/734707,gold,"var ctp1 = "" 熊本のニュース"";",0.872809,0.945743,0.941882,0.146670,0.238188,-0.1108,0.121332,0.009689,-0.009051,0.037003
1206,/articles/835296,gold,"var ctp1 = "" 熊本のニュース"";",-0.135449,0.312942,0.569425,-0.288438,-0.143248,-0.1108,0.121332,0.009689,-0.009051,0.037003


Keyを数値化する

In [106]:
ar = pd.get_dummies(ar, columns=['key'], prefix='key', prefix_sep='-')
ar.drop(columns = ['genre', 'key-xxx'], inplace=True)

In [112]:
#URLを数値のみの表示にする

for i in range(len(ar)):
    ar['url'][i] = ar['url'][i].split('/articles/')[1]

/var/folders/fz/x8gfkjkj2y97_snsf3gnnrlr0000gn/T/ipykernel_42201/2095117837.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ar['url'][i] = ar['url'][i].split('/articles/')[1]


In [115]:
#列[url]をインデックスに
ar = ar.set_index('url')

In [119]:
from sklearn.decomposition import PCA

model_pca = PCA(n_components=6)
vecs_list = model_pca.fit_transform(ar)

for i in range(6):
    ar[f'article_{i}'] = 0.0

for i in range(len(vecs_list)):
    for j in range(6):
        ar[f'article_{j}'][i] = vecs_list[i][j]

/var/folders/fz/x8gfkjkj2y97_snsf3gnnrlr0000gn/T/ipykernel_42201/3245679776.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ar[f'article_{j}'][i] = vecs_list[i][j]


In [121]:
ar = ar.filter(regex='^article')

In [125]:
# CSV ファイル (employee.csv) として出力
ar.to_csv("ar_202211.csv", header=False, sep='\t', float_format='%.6f')